In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from utils.preprocessing import hu_data, convert_signed_4bit_to_hu, convert_to_signed_4bit
import json

from utils.vis import plot_images_nib, plot_images, analyze_inhale_exhale, plot_slice
from utils.segment import apply_lungmask_binary, apply_lungmask_thresholded, convert_nrrd_to_nii_gz


In [2]:
DATA_PATH = './data'
RESULT_PATH = './results'
PARAMETER_PATH = './parameter'
ELASTIX_PATH = './elastix'
DATA_PATH_4BIT = './data_4bit'
MASK_PATH = './mask'
dataset_info = json.load(open("./data/dataset_info.json"))
save = False
vis = False

# Loop for Images

In [6]:
for key, value in dataset_info.items():
    print(f"Processing {key}")
    
    ## Read the images
    inhale_image_path = os.path.join(DATA_PATH, key, f'{key}_iBHCT.nii.gz')
    exhale_image_path = os.path.join(DATA_PATH, key, f'{key}_eBHCT.nii.gz')
    lung_inhale_export_path = os.path.join(MASK_PATH, key)
    lung_exhale_export_path = os.path.join(MASK_PATH, key)
        
    inhale_image = nib.load(inhale_image_path)
    inhale_data = inhale_image.get_fdata()
    exhale_image = nib.load(exhale_image_path)
    exhale_data = exhale_image.get_fdata()

    ## Preprocess the images
    hu_inhale = hu_data(inhale_data)
    hu_exhale = hu_data(exhale_data)

    inhale_bit_4 = convert_to_signed_4bit(hu_inhale)
    exhale_bit_4 = convert_to_signed_4bit(hu_exhale)

    inhale_bit_4_hu = convert_signed_4bit_to_hu(inhale_bit_4)
    exhale_bit_4_hu = convert_signed_4bit_to_hu(exhale_bit_4)

    lung_inhale = nib.load(os.path.join(lung_inhale_export_path, f'{key}_iBHCT_lung_mask.nii.gz')).get_fdata().astype(np.uint8)
    lung_exhale = nib.load(os.path.join(lung_exhale_export_path, f'{key}_eBHCT_lung_mask.nii.gz')).get_fdata().astype(np.uint8)

    lung_inhale_threholded = apply_lungmask_thresholded(inhale_data)
    lung_exhale_threholded = apply_lungmask_thresholded(exhale_data)

    lung_inhale_combined = np.bitwise_and(lung_inhale, lung_inhale_threholded)
    lung_exhale_combined = np.bitwise_and(lung_exhale, lung_exhale_threholded)

    # Export 4-bit hu images
    inhale_4bit_export_path = os.path.join(DATA_PATH_4BIT, key)
    exhale_4bit_export_path = os.path.join(DATA_PATH_4BIT, key)
    os.makedirs(inhale_4bit_export_path, exist_ok=True)
    os.makedirs(exhale_4bit_export_path, exist_ok=True)
    inhale_4bit_hu = nib.Nifti1Image(inhale_bit_4_hu, inhale_image.affine)
    exhale_4bit_hu = nib.Nifti1Image(exhale_bit_4_hu, exhale_image.affine)
    nib.save(inhale_4bit_hu, os.path.join(inhale_4bit_export_path, f'{key}_iBHCT_4bit.nii.gz'))
    nib.save(exhale_4bit_hu, os.path.join(exhale_4bit_export_path, f'{key}_eBHCT_4bit.nii.gz'))

    ## Export the lung masks
    lung_inhale_export_path = os.path.join(MASK_PATH, key)
    lung_exhale_export_path = os.path.join(MASK_PATH, key)
    os.makedirs(lung_inhale_export_path, exist_ok=True)
    os.makedirs(lung_exhale_export_path, exist_ok=True)
    lung_inhale_nib = nib.Nifti1Image(lung_inhale, inhale_image.affine)
    lung_exhale_nib = nib.Nifti1Image(lung_exhale, exhale_image.affine)
    nib.save(lung_inhale_nib, os.path.join(lung_inhale_export_path, f'{key}_iBHCT_lung_mask.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)
    nib.save(lung_exhale_nib, os.path.join(lung_exhale_export_path, f'{key}_eBHCT_lung_mask.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)

    ## Export the thresholded lung masks
    lung_inhale_thresholded_export_path = os.path.join(MASK_PATH, key)
    lung_exhale_thresholded_export_path = os.path.join(MASK_PATH, key)
    os.makedirs(lung_inhale_thresholded_export_path, exist_ok=True)
    os.makedirs(lung_exhale_thresholded_export_path, exist_ok=True)
    lung_inhale_thresholded_nib = nib.Nifti1Image(lung_inhale_threholded, inhale_image.affine)
    lung_exhale_thresholded_nib = nib.Nifti1Image(lung_exhale_threholded, exhale_image.affine)
    nib.save(lung_inhale_thresholded_nib, os.path.join(lung_inhale_thresholded_export_path, f'{key}_iBHCT_lung_mask_thresholded.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)    
    nib.save(lung_exhale_thresholded_nib, os.path.join(lung_exhale_thresholded_export_path, f'{key}_eBHCT_lung_mask_thresholded.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)   

    ## Export the combined lung masks
    lung_inhale_combined_export_path = os.path.join(MASK_PATH, key)
    lung_exhale_combined_export_path = os.path.join(MASK_PATH, key)
    os.makedirs(lung_inhale_combined_export_path, exist_ok=True)
    os.makedirs(lung_exhale_combined_export_path, exist_ok=True)
    lung_inhale_combined_nib = nib.Nifti1Image(lung_inhale_combined, inhale_image.affine)
    lung_exhale_combined_nib = nib.Nifti1Image(lung_exhale_combined, exhale_image.affine)
    nib.save(lung_inhale_combined_nib, os.path.join(lung_inhale_combined_export_path, f'{key}_iBHCT_lung_mask_combined.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)
    nib.save(lung_exhale_combined_nib, os.path.join(lung_exhale_combined_export_path, f'{key}_eBHCT_lung_mask_combined.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)


    

Processing copd1
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
Processing copd2
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
**********************

In [8]:
for key, value in dataset_info.items():
    print(f"Processing {key}")
    
    ## Read the images
    inhale_image_path = os.path.join(DATA_PATH, key, f'{key}_iBHCT.nii.gz')
    exhale_image_path = os.path.join(DATA_PATH, key, f'{key}_eBHCT.nii.gz')
    
    inhale_mask_path = os.path.join(MASK_PATH, key, 'Lungsegmentation_inhale.nrrd')
    exhale_mask_path = os.path.join(MASK_PATH, key, 'Lungsegmentation_exhale.nrrd')

    inhale_mask_with_trachea, inhale_mask_without_trachea = convert_nrrd_to_nii_gz(inhale_mask_path, inhale_image_path)
    exhale_mask_with_trachea, exhale_mask_without_trachea = convert_nrrd_to_nii_gz(exhale_mask_path, exhale_image_path)
    ## Export the thresholded lung masks

    inhale_mask_with_trachea_export_path = os.path.join(MASK_PATH, key)
    exhale_mask_with_trachea_export_path = os.path.join(MASK_PATH, key)
    os.makedirs(inhale_mask_with_trachea_export_path, exist_ok=True)
    os.makedirs(exhale_mask_with_trachea_export_path, exist_ok=True)
    nib.save(inhale_mask_with_trachea, os.path.join(inhale_mask_with_trachea_export_path, f'{key}_iBHCT_lung_mask_with_trachea.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)
    nib.save(exhale_mask_with_trachea, os.path.join(exhale_mask_with_trachea_export_path, f'{key}_eBHCT_lung_mask_with_trachea.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)

    ## Export the combined lung masks
    inhale_mask_without_trachea_export_path = os.path.join(MASK_PATH, key)
    exhale_mask_without_trachea_export_path = os.path.join(MASK_PATH, key)
    os.makedirs(inhale_mask_without_trachea_export_path, exist_ok=True)
    os.makedirs(exhale_mask_without_trachea_export_path, exist_ok=True)
    nib.save(inhale_mask_without_trachea, os.path.join(inhale_mask_without_trachea_export_path, f'{key}_iBHCT_lung_mask_without_trachea.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)
    nib.save(exhale_mask_without_trachea, os.path.join(exhale_mask_without_trachea_export_path, f'{key}_eBHCT_lung_mask_without_trachea.nii.gz'))
    print('*'*50, 'lung mask saved', '*'*50)
    

Processing copd1
NIfTI image created but not saved (no output path provided).
NIfTI image created but not saved (no output path provided).
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
Processing copd2
NIfTI image created but not saved (no output path provided).
NIfTI image created but not saved (no output path provided).
************************************************** lung mask saved **************************************************
************************************************** lung mask saved **************************************************
**************